# Light GBM

In [1]:
import os
import datetime
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
if 'current_dir' in locals():
    print("Directory has been set before:")
else:
    current_dir = os.getcwd()[:-9] 
print(current_dir)
os.chdir(current_dir + '/scripts')
import preprocessing
os.chdir(current_dir)
submission = pd.read_csv(current_dir + 'submissions/submission_v1.csv') #submission as template (version is not important)

C:\Users\suhai\Downloads\Customer-Loyalty\


In [8]:
train_x, train_l, testset_x = preprocessing.load_data(features="custom", custom_features=["train_feature_fam.csv","test_feature_fam.csv"])

C:\Users\suhai\Downloads\Customer-Loyalty


In [12]:
train_x, train_l, testset_x = preprocessing.load_data(features="all")

C:\Users\suhai\Downloads\Customer-Loyalty
this is a test feature : test_feature_avg_purchases.csv
this is a test feature : test_feature_fam.csv
this is a test feature : test_feature_feature_1_is_1.csv
this is a test feature : test_feature_feature_1_is_2.csv
this is a test feature : test_feature_feature_1_is_3.csv
this is a test feature : test_feature_feature_1_is_4.csv
this is a test feature : test_feature_feature_1_is_5.csv
this is a test feature : test_feature_feature_2_is_1.csv
this is a test feature : test_feature_feature_2_is_2.csv
this is a test feature : test_feature_feature_2_is_3.csv
this is a train feature : train_feature_avg_purchases.csv
this is a train feature : train_feature_fam.csv
this is a train feature : train_feature_feature_1_is_1.csv
this is a train feature : train_feature_feature_1_is_2.csv
this is a train feature : train_feature_feature_1_is_3.csv
this is a train feature : train_feature_feature_1_is_4.csv
this is a train feature : train_feature_feature_1_is_5.csv

In [13]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
# Prepare data for training
X = train_x
y = train_l

# Split data with kfold
kfolds = KFold(n_splits=8, shuffle=True, random_state=2018)

oof_preds = np.zeros(X.shape[0])
sub_preds = np.zeros(testset_x.shape[0])

for n_fold, (trn_idx, val_idx) in enumerate(kfolds.split(X, y)):
    X_train, y_train = X[trn_idx], y[trn_idx]
    X_valid, y_valid = X[val_idx], y[val_idx]
    
    # LightGBM Regressor estimator
    model = lgb.LGBMRegressor(
        num_leaves = 33,
        learning_rate = 0.05,
        n_estimators = 1000,
        subsample = .9,
        colsample_bytree = .9,
        random_state = 100,
        booster = "gbdt",
        eval_metric = "rmse",
        nthread = 10,
        nrounds = 1500,
        max_depth = 7,
        max_bin = 500,
    )
    
    # Fit
    model.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        verbose=None, eval_metric='rmse',
        early_stopping_rounds=100
    )

    oof_preds[val_idx] = model.predict(X_valid, num_iteration=model.best_iteration_)
    test_preds = model.predict(testset_x, num_iteration=model.best_iteration_)
    sub_preds += test_preds / kfolds.n_splits
    predictions = [round(value) for value in sub_preds]
    submission.target = predictions
    
mean_squared_error(y, oof_preds) ** .5

3.840522082696508

In [14]:
# run this after previous cel to see the submission
submission.target.value_counts()

-0.0    86065
-1.0    37482
-2.0       63
-3.0       12
-4.0        1
Name: target, dtype: int64

In [15]:
# submission.target = submission_predictions

for i in range(1,100,1):
    if os.path.isfile('submissions/submission_v0-00' + str(i) + '.csv') == False:
        submission.to_csv('submissions/submission_v0-00' + str(i) + '.csv', index = False)
        print('Submission was saved under the name:' + '\033[1m' ' submission_v0-00' + str(i) + '.csv' + '\033[0;0m')
        break

# [Errno 13] Permission denied: 'submissions/submission_v0-001.csv'
#        means that the file is still opened somewhere

Submission was saved under the name: submission_v0-006.csv
